In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from niwidgets import NiftiWidget
import tensorflow as tf
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import os
import tensorflow as tf

os.environ["AUTOGRAPH_VERBOSITY"] = '0'
# Verbosity is now 5

tf.autograph.set_verbosity(0)
from tensorflow.python.client import device_lib

/home/jain0000/miniconda3/envs/segment2/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
import math
#import cv2
#import random
#from skimage.transform import rescale, resize, downscale_local_mean,rotate
import numpy as np
#from skimage.util import montage, crop
#from tensorflow.keras.models import load_model
from skimage.transform import rescale, resize, downscale_local_mean,rotate
#from skimage import data, color
#from scipy import ndimage
#from scipy import misc
from numpy import fliplr

import numpy as np
#from skimage.measure import label


In [3]:
from config import *
from modelIO import *

In [4]:
class weighted_bce(Loss):
      def call(self, y_true, y_pred, weight1=0.1, weight0=1 ):
       # wmap =unet_weight_map(y, wc=None, w0 = 1, sigma = 2)
        y_pred = ops.convert_to_tensor_v2(y_pred)
        y_true = gen_math_ops.cast(y_true, y_pred.dtype)
        y_true = tf.keras.backend.clip(y_true, tf.keras.backend.epsilon(), 1-tf.keras.backend.epsilon())
        y_pred = tf.keras.backend.clip(y_pred, tf.keras.backend.epsilon(), 1-tf.keras.backend.epsilon())
        logloss = -(y_true * tf.keras.backend.log(y_pred) * weight0 + (1 - y_true) * tf.keras.backend.log(1 - y_pred) * weight1 )
        return tf.keras.backend.mean( logloss, axis=-1)
    
class Dice(Loss):
      def call(self, y_true, y_pred, weight1=0.1, weight0=1 ):
        y_pred = ops.convert_to_tensor_v2(y_pred)
        y_true = gen_math_ops.cast(y_true, y_pred.dtype)
        y_true = tf.keras.backend.clip(y_true, tf.keras.backend.epsilon(), 1-tf.keras.backend.epsilon())
        y_pred = tf.keras.backend.clip(y_pred, tf.keras.backend.epsilon(), 1-tf.keras.backend.epsilon())
        
        intersection = tf.keras.backend.sum(tf.keras.backend.abs(y_true * y_pred), axis=-1)
        numerator = (2. * intersection + 1) 
        denominator = (tf.keras.backend.sum(tf.keras.backend.square(y_true),-1) + tf.keras.backend.sum(tf.keras.backend.square(y_pred),-1) + 1)
        return 1 - (numerator/denominator)


from tensorflow.keras.metrics import MeanIoU,BinaryCrossentropy,Accuracy,Precision,Recall,AUC
@tf.autograph.experimental.do_not_convert
def print_metrics(TestType,mask,pred_mask,model,niiImage ):
    
   
    m2 = AUC()
    m2.update_state(mask.flatten(), pred_mask.flatten())

    m4 = Accuracy()
    m4.update_state(mask.flatten(), pred_mask.flatten())

    m5 = Precision()
    m5.update_state(mask.flatten(), pred_mask.flatten())

    m7 = Recall()
    m7.update_state(mask.flatten(), pred_mask.flatten())
    
    m8 = tf.keras.metrics.FalsePositives()
    m8.update_state(mask, pred_mask)
    print("false positives",m8.result().numpy())
    
    m9= tf.keras.metrics.FalseNegatives()
    m9.update_state(mask, pred_mask)
    print("false negatives",m9.result().numpy())
    
    m10 = tf.keras.metrics.TruePositives()
    m10.update_state(mask, pred_mask)
    print("true positives",m10.result().numpy())

    m11= tf.keras.metrics.TrueNegatives()
    m11.update_state(mask, pred_mask)
    print("true negatives",m11.result().numpy())
  #  from scipy.spatial.distance import directed_hausdorff
  #  m5 = directed_hausdorff(mask, pred_mask)[0]
   # m5.update_state(mask.flatten(), pred_mask.flatten())
    
    m =( 2 *  m10.result().numpy())/ ( m10.result().numpy() + m8.result().numpy() +  m10.result().numpy() +  m9.result().numpy())
    m6 = m11.result().numpy()/(m11.result().numpy()+m8.result().numpy())
  #  m6.update_state(mask.flatten(), pred_mask.flatten())
   # print("f1 score", m7.result().numpy())
    
    print("TestType",'1',"DICE",m,"Accuracy",m4.result().numpy(),"Specificity",m6,"AUC",m2.result().numpy())
    
     # create logger
    #logging.disable(logging.WARNING)    
    logging.info(' Test : %s, Model: %s, Type: %s , sample : %s, Accuracy : %s ,DICE : %s , Specificity: %s, AUC: %s','Segmentation2D', TestType,model, niiImage ,m4.result().numpy(),m,
         m6,m2.result().numpy())
    # define a Handler which writes INFO messages or higher to the sys.stderr
    return None

def normalizeImageIntensityRange(img):
    
    img[img < HOUNSFIELD_MIN] = HOUNSFIELD_MIN
    img[img > HOUNSFIELD_MAX] = HOUNSFIELD_MAX
    return (img - HOUNSFIELD_MIN) / HOUNSFIELD_RANGE
def get_largest_component(image):
    """
    get the largest component from 2D or 3D binary image
    image: nd array
    """
    dim = len(image.shape)
    if(image.sum() == 0 ):
        print('the largest component is null')
        return image
    if(dim == 2):
        s = ndimage.generate_binary_structure(2,1)
    elif(dim == 3):
        s = ndimage.generate_binary_structure(3,1)
    else:
        raise ValueError("the dimension number should be 2 or 3")
    labeled_array, numpatches = ndimage.label(image, s)
    sizes = ndimage.sum(image, labeled_array, range(1, numpatches + 1))
    max_label = np.where(sizes == sizes.max())[0] + 1
    output = np.asarray(labeled_array == max_label, np.double)
    return  output 


In [5]:
dataDirectory = data_dir_test
dirs = os.listdir(os.path.join(data_dir_test_image))


In [6]:
import logging
from skimage.measure import marching_cubes_lewiner
import meshplot as mp
def test(task,data_type):
    #os.listdir(os.path.join(data_dir_test+str('image')))
    #dirs = os.listdir(data_dir_train_image)
    maskArr=[]
    predMaskArr=[]
    for d in os.listdir(data_dir_test_image)  :
        #if d in ['mandible-Right-Molar','mandible-Right-Canine','maxilla-Right-Molar']:
        for niiImage in os.listdir(os.path.join(data_dir_test_image,d)):
            model = load('classification')
            #model2= load('classification')
        #    model3 = load('segmentation/molar')
        #    model4= load('segmentation/canine')
          #  print(os.path.join(data_dir_test_mask,d, niiImage))
           # print(os.path.isfile(os.path.join(data_dir_test_mask,d, niiImage)))
            #if(os.path.isfile(os.path.join(data_dir_test_mask,d, niiImage))):
            img = nib.load( os.path.join(data_dir_test_image,d, niiImage))
           # inputs = normalizeImageIntensityRange(img.get_fdata() ) 
            print(inputs.shape)
            #inputs = exposure.equalize_hist(inputs)
            mask = np.round(nib.load( os.path.join(data_dir_test_mask,d, niiImage)).get_fdata()) 

            print(niiImage)

            logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s',
                    filename='myapp.log',
                    filemode='a')


            console = logging.StreamHandler()
            console.setLevel(logging.INFO)
            # set a format which is simpler for console use
            formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
            # tell the handler to use this format
            console.setFormatter(formatter)
            # add the handler to the root logger
            logging.getLogger('').addHandler(console)

           # img_np = np.rollaxis(np.array(np.array([np.array([np.array(inputs)]) ])),0,5)
            if task=='segmentation2d':
             # print(resize_image.shape)
                pred = predictVolume(model,inputs) 
                #i = resize(pred, (40, 40, 40), anti_aliasing=True)
                pred1 = np.rollaxis(np.array(np.array([np.array([np.array(pred)]) ])),0,5)
            else:
                shape = (IMAGE_HEIGHT, IMAGE_WIDTH,IMAGE_DEPTH)
                resize_image = resize(inputs, shape, anti_aliasing=True)
                resize_mask = resize(mask, shape, anti_aliasing=True)    
                img_np = np.rollaxis(np.array(np.array([np.array([np.array(resize_image)]) ])),0,5)
               # @tf.autograph.experimental.do_not_convert
                #@tf.autograph.experimental.do_not_convert
                predsc = (model.predict(img_np)).astype(float)
                pred1 = predsc[0]
                
                #fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, pred1)
            class_no = predsc[1] 
            print("Predicted : ", class_no," True class:  ", d )
            print(pred1.shape)
            pred_mask  = resize_data(pred1[0][:,:,:,0],mask.shape) 
            
           # y=pred_mask
         #   y = get_largest_component(np.round(pred_mask))
        #    img = nib.Nifti1Image(y,  img.affine)
         #   if os.path.isdir(os.path.join(data_dir_test,'segmented-volume',d)):
       #         nib.save(img, os.path.join(data_dir_test,'segmented-volume',d, niiImage))  
       #     else:
       #         directory = os.mkdir(os.path.join(data_dir_test,'segmented-volume',d))
       #         nib.save(img, os.path.join(data_dir_test,'segmented-volume',d, niiImage))  

            #y = get_largest_component(np.round(pred_mask))
           # maskArr.append(mask)
           # predMaskArr.append(y)
         #   x = print_metrics('Sparse',mask,(y),str(d),niiImage)

          #  display(inputs,pred_mask)
         #   vertices,faces,_,_ = marching_cubes_lewiner(y)

        #    mp.plot(vertices, faces, return_plot=False)
    
    
        #    from sklearn.metrics import roc_curve
           # print(roc_curve(mask.flatten(), y.flatten()))
          #  fpr_keras, tpr_keras, thresholds_keras =(np.array([0.        , 0.00239111, 1.        ]), np.array([0.        , 0.90658531, 1.        ]), np.array([2., 1., 0.]))

 #roc_curve(mask.flatten(), y.flatten())
          #  from sklearn.metrics import auc
        #    auc_keras = auc(fpr_keras, tpr_keras)

        #    plt.figure(1)
        #    plt.plot([0, 1], [0, 1], 'k--')
       #     plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
       #     plt.xlabel('False positive rate')
       #     plt.ylabel('True positive rate')
       #     plt.title('AUC curve')
       #     plt.legend(loc='best')
       #     plt.show()
    
    return (img,pred_mask)



In [ ]:
print(tf.__version__)
with tf.device('/device:gpu:2'):
    test('classification',['mandible-Right-Molar','mandible-Right-Canine'])